In [ ]:
# Top stackoverflow tags
Dataset: https://www.kaggle.com/iancuv/stackoverflow-question-favourites

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType, TimestampType

In [4]:
spark = SparkSession.builder.appName("stackoverflow_tags").master("local[*]").getOrCreate()

In [9]:
questions_schema = StructType([
    StructField("Id", IntegerType()), 
    StructField("CreationDate", TimestampType()),
    StructField("ClosedDate", TimestampType()),
    StructField("DeletionDate", TimestampType()),
    StructField("Score", IntegerType()),
    StructField("OwnerUserId", IntegerType()),
    StructField("AnswerCount", IntegerType()),
])

question_tag_schema = StructType([
    StructField("Id", IntegerType()), 
    StructField("Tag", StringType()),
])

In [14]:
questions = spark.read.csv("data/questions.csv", schema=questions_schema, header="true", sep=",",  nullValue = "NA")
question_tags = spark.read.csv("data/question_tags.csv", schema=question_tag_schema, header="true", sep=",", nullValue="NA")

In [16]:
questions.printSchema()
question_tags.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- DeletionDate: timestamp (nullable = true)
 |-- Score: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)

root
 |-- Id: integer (nullable = true)
 |-- Tag: string (nullable = true)



In [27]:
questions.createOrReplaceTempView("questions")
question_tags.createOrReplaceTempView("question_tags")

+---+---------------+
| Id|            Tag|
+---+---------------+
|  1|           data|
|  4|             c#|
|  4|       winforms|
|  4|type-conversion|
|  4|        decimal|
+---+---------------+
only showing top 5 rows



In [38]:
top_tags=spark.sql("""
SELECT 
Tag, COUNT(1) as cnt
FROM questions q
JOIN question_tags qt ON q.Id = qt.Id
GROUP BY Tag
""")


In [39]:
top_tags.write.parquet("data/parquet/top_tags_sql.parquet")